<a href="https://colab.research.google.com/github/yeon-do/ATL_2_1/blob/main/mission1_2_albumentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image
import numpy as np
import time
import zipfile
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# 이미지가 저장된 경로 설정
image_dir = '/content/drive/MyDrive/mission_data/training_image'   # train

image_dir2 = '/content/drive/MyDrive/mission_data/validation_image'    # val

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# 파일명을 파싱하여 성별과 스타일 정보를 추출하는 함수
def parse_filename(filename):
    # 파일명에서 확장자를 제거한 후 "_"로 분할
    file_parts = filename.split('.')[0].split('_')
    if len(file_parts) < 4:
        print(f"Filename format incorrect: {filename}")
        return None, None  # 파싱 실패 시 None 반환
    # 성별과 스타일 추출
    gender = file_parts[-1]  # 성별 (M or W)
    style = file_parts[-2]  # 스타일
    return gender, style

# 폴더 내 모든 파일을 재귀적으로 탐색하며 성별 및 스타일 통계를 계산하는 함수
def calculate_statistics(image_dir):
    # 성별과 스타일별 카운트를 저장할 딕셔너리
    stats = defaultdict(lambda: defaultdict(int))

    # 이미지 폴더 내의 모든 파일을 재귀적으로 탐색
    for root, _, files in os.walk(image_dir):
        for filename in files:
            if filename.endswith(".jpg"):
                gender, style = parse_filename(filename)
                if gender and style:  # 성별과 스타일이 파싱된 경우에만 처리
                    stats[gender][style] += 1

    # 성별, 스타일, 이미지 수로 구성된 DataFrame 생성
    data = []
    for gender, styles in stats.items():
        for style, count in styles.items():
            if gender and style:  # 파싱된 값이 유효할 때만 추가
                data.append([gender, style, count])

    if not data:
        print(f"No valid data found in directory: {image_dir}")
        return pd.DataFrame(columns=["Gender", "Style", "Count"])

    df = pd.DataFrame(data, columns=["Gender", "Style", "Count"])
    return df

train_stats = calculate_statistics(image_dir)


val_stats = calculate_statistics(image_dir2)

# 결과 출력
display(train_stats)

print("\nValidation Statistics:")
val_stats

,Gender,Style,Count
0,M,hippie,260
1,M,hiphop,274
2,M,normcore,364
3,M,bold,268
4,M,ivy,237
5,M,mods,269
6,M,metrosexual,278
7,M,sportivecasual,298
8,W,normcore,153
9,W,hippie,91



Validation Statistics:


,Gender,Style,Count
0,W,cityglam,18
1,W,feminine,44
2,W,kitsch,22
3,W,minimal,35
4,W,military,9
5,W,sportivecasual,48
6,W,powersuit,34
7,W,hippie,14
8,W,genderless,12
9,W,oriental,18


## 변경된 코드

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from tqdm import tqdm  # tqdm을 사용하여 진행 상황 표시

# 1. 커스텀 데이터셋 클래스 정의 (합쳐진 데이터셋)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        """
        img_dir: 이미지가 합쳐져 있는 디렉토리
        transform: 이미지 전처리 변환
        """
        self.img_dir = img_dir
        self.transform = transform

        # 이미지 파일 목록 불러오기 (성별과 상관없이 모두)
        self.images = [img for img in os.listdir(img_dir) if img.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 이미지 경로와 파일명
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = Image.open(img_path).convert("RGB")  # 이미지를 RGB로 변환

        # 파일 이름으로 성별 레이블 결정 (예: 파일 이름에 'W'가 포함되면 여성, 'M'이면 남성)
        if 'W' in self.images[idx]:
            gender_label = 0  # W로 시작하는 파일은 성별 0 (여성)
        elif 'M' in self.images[idx]:
            gender_label = 1  # M로 시작하는 파일은 성별 1 (남성)
        else:
            raise ValueError("파일 이름에 성별을 구분할 수 있는 정보가 없습니다.")

        # 스타일 레이블을 임의로 지정 (2개의 스타일로 가정)
        style_label = torch.randint(0, 2, (1,)).item()

        # 이미지 전처리 적용
        if self.transform:
            image = self.transform(image)

        return image, gender_label, style_label

# 2. 데이터 전처리 정의
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 3. 데이터셋 로드 (이미지들이 하나의 디렉토리에 있는 경우)
train_dataset = CustomImageDataset(image_dir, transform=train_transforms)
val_dataset = CustomImageDataset(image_dir2, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 4. ResNet-18 모델 설정 (다중 출력 분류)
class MultiOutputResNet(nn.Module):
    def __init__(self, num_gender_classes, num_style_classes):
        super(MultiOutputResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)

        # fully connected 레이어 삭제 및 성별/스타일 레이어 추가
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # 기존 fully connected 레이어 비활성화
        self.fc_gender = nn.Linear(in_features, num_gender_classes)  # 성별 분류 레이어
        self.fc_style = nn.Linear(in_features, num_style_classes)    # 스타일 분류 레이어

    def forward(self, x):
        features = self.resnet(x)  # ResNet을 통해 특징 추출
        gender_output = self.fc_gender(features)  # 성별 분류
        style_output = self.fc_style(features)    # 스타일 분류
        return gender_output, style_output

# 5. 모델 생성
model = MultiOutputResNet(num_gender_classes=2, num_style_classes=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion_gender = nn.CrossEntropyLoss()
criterion_style = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 6. 학습 과정 정의 (진행 퍼센트 표시)
def train(model, train_loader, criterion_gender, criterion_style, optimizer):
    model.train()
    running_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    # tqdm을 사용하여 진행 상황 표시
    train_loader_tqdm = tqdm(train_loader, desc="Training", unit="batch")

    for images, gender_labels, style_labels in train_loader_tqdm:
        images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

        optimizer.zero_grad()

        # 순전파
        gender_outputs, style_outputs = model(images)

        # 성별과 스타일에 대한 손실 각각 계산
        loss_gender = criterion_gender(gender_outputs, gender_labels)
        loss_style = criterion_style(style_outputs, style_labels)
        loss = loss_gender + loss_style  # 총 손실은 두 손실의 합

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted_gender = torch.max(gender_outputs, 1)
        _, predicted_style = torch.max(style_outputs, 1)

        total += gender_labels.size(0)
        correct_gender += (predicted_gender == gender_labels).sum().item()
        correct_style += (predicted_style == style_labels).sum().item()

        # 진행 상태를 표시할 때도 정확도나 손실 등을 업데이트
        train_loader_tqdm.set_postfix(loss=loss.item(), gender_acc=correct_gender/total, style_acc=correct_style/total)

    avg_loss = running_loss / len(train_loader)
    gender_accuracy = correct_gender / total
    style_accuracy = correct_style / total
    return avg_loss, gender_accuracy, style_accuracy

# 7. 검증 과정 정의
def validate(model, val_loader, criterion_gender, criterion_style):
    model.eval()
    val_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    # tqdm을 사용하여 진행 상황 표시
    val_loader_tqdm = tqdm(val_loader, desc="Validation", unit="batch")

    with torch.no_grad():
        for images, gender_labels, style_labels in val_loader_tqdm:
            images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

            gender_outputs, style_outputs = model(images)

            loss_gender = criterion_gender(gender_outputs, gender_labels)
            loss_style = criterion_style(style_outputs, style_labels)
            loss = loss_gender + loss_style

            val_loss += loss.item()
            _, predicted_gender = torch.max(gender_outputs, 1)
            _, predicted_style = torch.max(style_outputs, 1)

            total += gender_labels.size(0)
            correct_gender += (predicted_gender == gender_labels).sum().item()
            correct_style += (predicted_style == style_labels).sum().item()

            # 진행 상태를 표시할 때도 정확도나 손실 등을 업데이트
            val_loader_tqdm.set_postfix(loss=loss.item(), gender_acc=correct_gender/total, style_acc=correct_style/total)

    avg_val_loss = val_loss / len(val_loader)
    gender_accuracy = correct_gender / total
    style_accuracy = correct_style / total
    return avg_val_loss, gender_accuracy, style_accuracy

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# 8. 모델 학습 실행
def train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch [{epoch+1}/{num_epochs}]")

        # 학습 단계
        train_loss, train_gender_acc, train_style_acc = train(
            model, train_loader, criterion_gender, criterion_style, optimizer
        )

        # 검증 단계
        val_loss, val_gender_acc, val_style_acc = validate(
            model, val_loader, criterion_gender, criterion_style
        )

        # 에폭별 결과 출력
        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {train_loss:.4f}, Train Gender Acc: {train_gender_acc:.4f}, Train Style Acc: {train_style_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Gender Acc: {val_gender_acc:.4f}, Validation Style Acc: {val_style_acc:.4f}")

In [ ]:
# 9. 학습 실행
train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs=10)

Epoch [1/10]


Validation: 100%|██████████| 30/30 [13:54<00:00, 27.81s/batch, gender_acc=0.993, loss=0.722, style_acc=0.524]


Epoch [1/10] - Train Loss: 0.8271, Train Gender Acc: 0.9892, Train Style Acc: 0.4968
Validation Loss: 0.7346, Validation Gender Acc: 0.9926, Validation Style Acc: 0.5237
Epoch [2/10]


Validation: 100%|██████████| 30/30 [05:42<00:00, 11.43s/batch, gender_acc=0.993, loss=0.774, style_acc=0.507]


Epoch [2/10] - Train Loss: 0.7399, Train Gender Acc: 0.9946, Train Style Acc: 0.5064
Validation Loss: 0.7510, Validation Gender Acc: 0.9926, Validation Style Acc: 0.5068
Epoch [3/10]


...

Epoch [6/10] - Train Loss: 0.7332, Train Gender Acc: 0.9946, Train Style Acc: 0.5007
Validation Loss: 0.7291, Validation Gender Acc: 0.9926, Validation Style Acc: 0.5026
Epoch [7/10]


Validation: 100%|██████████| 30/30 [05:37<00:00, 11.25s/batch, gender_acc=0.993, loss=0.736, style_acc=0.493]


Epoch [7/10] - Train Loss: 0.7272, Train Gender Acc: 0.9946, Train Style Acc: 0.5084
Validation Loss: 0.7274, Validation Gender Acc: 0.9926, Validation Style Acc: 0.4932
Epoch [8/10]


Training:   2%|▏         | 2/128 [00:47<50:03, 23.84s/batch, gender_acc=1, loss=0.72, style_acc=0.438]


KeyboardInterrupt: 

### try 5

optimizer 학습률 작게 조정, albumentations 모듈 변환기 사용해 이미지 증강, 정규화 기법 추가(drop out)

In [ ]:
!pip install albumentations
!pip install --upgrade albumentations[imgaug]  # 추가적인 이미지 증강 도구 사용 가능
!pip install opencv-python-headless  # OpenCV 설치 (Albumentations에서 필요)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.16
    Uninstalling albucore-0.0.16:
      Successfully uninstalled albucore-0.0.16
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.15
    Uninstalling albumentations-1.4.15:
      Successfully uninstalled albumentations-1.4.15


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from tqdm import tqdm  # tqdm을 사용하여 진행 상황 표시
import albumentations as A
from albumentations.pytorch import ToTensorV2


# 1. Albumentations를 이용한 데이터 전처리 및 증강
# 학습용 데이터 전처리 정의 (여러 증강 기법 포함)
train_transforms = A.Compose([
    A.Resize(224, 224),  # 크기 조정
    A.HorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    A.VerticalFlip(p=0.5),  # 상하 반전 추가
    A.RandomBrightnessContrast(p=0.2),  # 밝기 및 대비 조절
    A.Rotate(limit=30, p=0.5),  # -30도에서 30도까지 랜덤 회전
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.5),  # 이동, 확대/축소, 회전
    A.GaussianBlur(p=0.3),  # 가우시안 블러 추가
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),  # 색상 변화 추가
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet 정규화
    ToTensorV2(),  # PyTorch 텐서로 변환
])

# 검증용 데이터 전처리 정의 (증강 없이 정규화만 적용)
val_transforms = A.Compose([
    A.Resize(224, 224),  # 크기 조정
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet 정규화
    ToTensorV2(),  # PyTorch 텐서로 변환
])


# 2. 파일 이름을 이용한 클래스 및 레이블 구분
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [img for img in os.listdir(img_dir) if img.endswith('.jpg')]

        # 1. 데이터셋 내에서 고유한 스타일 문자열을 추출하여 정수로 매핑
        self.style_map = self._create_style_map()

        # **스타일 맵을 출력하여 확인**
        #print(f"Style Map: {self.style_map}")

    def _create_style_map(self):
        # 스타일별 문자열을 추출하여 고유한 스타일을 정수로 매핑하는 딕셔너리 생성
        style_set = set()  # 중복을 제거하기 위해 set 사용
        for img in self.images:
            parts = img.split('_')
            style_set.add(parts[3])  # 네 번째 항목이 스타일

        # 스타일 문자열을 정수로 매핑하는 딕셔너리 생성
        style_map = {style: idx for idx, style in enumerate(sorted(style_set))}
        return style_map

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      img_path = os.path.join(self.img_dir, self.images[idx])
      image = np.array(Image.open(img_path).convert("RGB"))

      filename = os.path.basename(img_path)
      parts = filename.split('_')

      # 성별 레이블 추출
      gender_label = 0 if parts[4].startswith('W') else 1  # 성별 레이블: W -> 0, M -> 1

      # 스타일 레이블 추출 (성별에 따라 다른 스타일 레이블을 정수로 변환)
      style_label_str = parts[3]
      style_label = self.style_map.get(style_label_str, -1)  # 문자열을 정수로 변환, 없으면 -1 반환

      # 남성 스타일은 0~7, 여성 스타일은 0~21 범위 내에 있어야 함
      if gender_label == 1:  # 남성
        if style_label > 7:  # 남성 스타일 범위를 벗어났는지 확인
            raise ValueError(f"Style '{style_label_str}' for male is out of bounds")
      else:  # 여성
        if style_label > 21:  # 여성 스타일 범위를 벗어났는지 확인
            raise ValueError(f"Style '{style_label_str}' for female is out of bounds")

      if style_label == -1:
        raise ValueError(f"Style '{style_label_str}' not found in style_map.")

      if self.transform:
          augmented = self.transform(image=image)
          image = augmented['image']

      return image, gender_label, style_label


# 3. 데이터셋 로드 (이미지들이 하나의 디렉토리에 있는 경우)
train_dataset = CustomImageDataset(image_dir, transform=train_transforms)
val_dataset = CustomImageDataset(image_dir2, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 4. ResNet-18 모델 설정 (다중 출력 분류)

class MultiOutputResNet(nn.Module):
    def __init__(self, num_gender_classes=2, num_male_styles=8, num_female_styles=22, dropout_prob=0.5):
        super(MultiOutputResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)

        # ResNet의 fully connected 레이어 삭제
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # 기존 fully connected 레이어 제거

        # Dropout 레이어 추가 (확률 50%)
        self.dropout = nn.Dropout(p=dropout_prob)

        # 성별 분류 레이어
        self.fc_gender = nn.Linear(in_features, num_gender_classes)  # 성별 분류 (2개 클래스: W, T)

        # 성별에 따른 스타일 분류 레이어
        self.fc_male_style = nn.Linear(in_features, num_male_styles)    # 남성 스타일 분류
        self.fc_female_style = nn.Linear(in_features, num_female_styles) # 여성 스타일 분류

    def forward(self, x):
        features = self.resnet(x)

        # Dropout 적용
        features = self.dropout(features)

        # 성별 분류
        gender_output = self.fc_gender(features)

        # 남성 스타일 분류
        male_style_output = self.fc_male_style(features)
        # 여성 스타일 분류
        female_style_output = self.fc_female_style(features)

        return gender_output, male_style_output, female_style_output


# 5. 모델 생성 및 가중치 초기화
#가중치 초기화 함수
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)  # Xavier 초기화 (Glorot 초기화)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)  # bias는 0으로 초기화

# 모델 생성 후 가중치 초기화 적용
num_male_styles = 8   # 남성 스타일 개수
num_female_styles = 22 # 여성 스타일 개수
model = MultiOutputResNet(num_gender_classes=2, num_male_styles=num_male_styles, num_female_styles=num_female_styles, dropout_prob=0.5)
model.apply(initialize_weights)  # 가중치 초기화 수행
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion_gender = nn.CrossEntropyLoss()
criterion_style = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

# 6. 학습 과정 정의 (진행 퍼센트 표시)
def train(model, train_loader, criterion_gender, criterion_style, optimizer):
    model.train()
    running_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    # tqdm을 사용하여 진행 상황 표시
    train_loader_tqdm = tqdm(train_loader, desc="Training", unit="batch")

    for images, gender_labels, style_labels in train_loader_tqdm:
        images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

        # **타겟 레이블 확인**
        #print(f"Gender Labels (Max): {gender_labels.max().item()}, Style Labels (Max): {style_labels.max().item()}")
        #print(f"Gender Labels (Min): {gender_labels.min().item()}, Style Labels (Min): {style_labels.min().item()}")


        optimizer.zero_grad()

        # 순전파
        gender_output, male_style_output, female_style_output = model(images)

        # 성별에 따른 손실 계산
        loss_gender = criterion_gender(gender_output, gender_labels)
        loss_style = 0

        #스타일에 따른 손실 계산
        for i in range(gender_labels.size(0)):
            if gender_labels[i] == 0:  # 여성일 때
                loss_style += criterion_style(female_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
            else:  # 남성일 때
                loss_style += criterion_style(male_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))

        loss = loss_gender + loss_style  # 총 손실은 성별과 스타일 손실의 합

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted_gender = torch.max(gender_output, 1)

        total += gender_labels.size(0)
        correct_gender += (predicted_gender == gender_labels).sum().item()


        # 진행 상태를 표시할 때도 정확도나 손실 등을 업데이트
        train_loader_tqdm.set_postfix(loss=loss.item(), gender_acc=correct_gender / total)

    avg_loss = running_loss / len(train_loader)
    gender_accuracy = correct_gender / total
    return avg_loss, gender_accuracy


# 7. 검증 과정 정의
def validate(model, val_loader, criterion_gender, criterion_style):
    model.eval()
    val_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    # tqdm을 사용하여 진행 상황 표시
    val_loader_tqdm = tqdm(val_loader, desc="Validation", unit="batch")

    with torch.no_grad():
        for images, gender_labels, style_labels in val_loader_tqdm:
            images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

            gender_output, male_style_output, female_style_output = model(images)

            loss_gender = criterion_gender(gender_output, gender_labels)
            loss_style = 0

            for i in range(gender_labels.size(0)):
                if gender_labels[i] == 0:
                    loss_style += criterion_style(female_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
                else:
                    loss_style += criterion_style(male_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))

            loss = loss_gender + loss_style
            val_loss += loss.item()

            _, predicted_gender = torch.max(gender_output, 1)

            total += gender_labels.size(0)
            correct_gender += (predicted_gender == gender_labels).sum().item()


            # 진행 상태를 표시할 때도 정확도나 손실 등을 업데이트
            val_loader_tqdm.set_postfix(loss=loss.item(), gender_acc=correct_gender / total)

    avg_val_loss = val_loss / len(val_loader)
    gender_accuracy = correct_gender / total
    return avg_val_loss, gender_accuracy

# 8. 학습 실행
    def train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs=10):
      for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        avg_loss, gender_acc = train(model, train_loader, criterion_gender, criterion_style, optimizer)
        print(f"Training - Loss: {avg_loss:.4f}, Gender Accuracy: {gender_acc:.4f}")

        avg_val_loss, val_gender_acc = validate(model, val_loader, criterion_gender, criterion_style)
        print(f"Validation - Loss: {avg_val_loss:.4f}, Gender Accuracy: {val_gender_acc:.4f}")

In [ ]:
# 9. 학습 및 검증 데이터 로더 설정 후 학습 실행
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs=10)

Epoch 1/10


Training:   0%|          | 0/128 [00:00<?, ?batch/s]


ValueError: Style 'mods' for male is out of bounds

### 최종1 with albumentations


In [6]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from tqdm import tqdm  # tqdm을 사용하여 진행 상황 표시
import albumentations as A
from albumentations.pytorch import ToTensorV2



# 1. Albumentations를 이용한 데이터 전처리 및 증강
# 학습용 데이터 전처리 정의 (여러 증강 기법 포함)
train_transforms = A.Compose([
    A.Resize(224, 224),  # 크기 조정
    A.HorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    A.VerticalFlip(p=0.5),  # 상하 반전 추가
    A.RandomBrightnessContrast(p=0.2),  # 밝기 및 대비 조절
    A.Rotate(limit=30, p=0.5),  # -30도에서 30도까지 랜덤 회전
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.5),  # 이동, 확대/축소, 회전
    A.GaussianBlur(p=0.3),  # 가우시안 블러 추가
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),  # 색상 변화 추가
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet 정규화
    ToTensorV2(),  # PyTorch 텐서로 변환
])

# 검증용 데이터 전처리 정의 (증강 없이 정규화만 적용)
val_transforms = A.Compose([
    A.Resize(224, 224),  # 크기 조정
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet 정규화
    ToTensorV2(),  # PyTorch 텐서로 변환
])


# 2. 파일 이름을 이용한 클래스 및 레이블 구분
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [img
        for img in os.listdir(img_dir) if img.endswith('.jpg')]

        # 남성과 여성의 스타일을 구분하여 각각 매핑
        self.male_style_map = self._create_male_style_map()
        self.female_style_map = self._create_female_style_map()

    def _create_male_style_map(self):
        # 남성 스타일 목록에 따라 정수로 매핑
        male_styles = ['sportivecasual', 'mods', 'hiphop', 'normcore', 'metrosexual', 'hippie', 'ivy', 'bold']
        male_style_map = {style: idx for idx, style in enumerate(male_styles)}
        return male_style_map

    def _create_female_style_map(self):
        # 여성 스타일 목록에 따라 정수로 매핑
        female_styles = [
            'sportivecasual', 'hiphop', 'oriental', 'normcore', 'lingerie', 'minimal', 'powersuit', 'space',
            'ecology', 'disco', 'hippie', 'genderless', 'feminine', 'cityglam', 'punk', 'classic', 'kitsch',
            'popart', 'bodyconscious', 'lounge', 'athleisure', 'grunge', 'military'
        ]
        female_style_map = {style: idx for idx, style in enumerate(female_styles)}
        return female_style_map

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = np.array(Image.open(img_path).convert("RGB"))

        filename = os.path.basename(img_path)
        parts = filename.split('_')

        # 성별 레이블 추출 (0: 여성, 1: 남성)
        gender_label = 0 if parts[4].startswith('W') else 1

        # 스타일 레이블 추출 (성별에 따라 다른 스타일 맵을 사용)
        style_label_str = parts[3]
        if gender_label == 1:  # 남성일 때
            style_label = self.male_style_map.get(style_label_str, -1)
            if style_label == -1:
                raise ValueError(f"Style '{style_label_str}' for male is not found in male_style_map.")
        else:  # 여성일 때
            style_label = self.female_style_map.get(style_label_str, -1)
            if style_label == -1:
                raise ValueError(f"Style '{style_label_str}' for female is not found in female_style_map.")

        # 이미지 전처리 및 증강 적용
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, gender_label, style_label



# 3. 데이터셋 로드 (이미지들이 하나의 디렉토리에 있는 경우)
train_dataset = CustomImageDataset(image_dir, transform=train_transforms)
val_dataset = CustomImageDataset(image_dir2, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 4. ResNet-18 모델 설정 (다중 출력 분류)

class MultiOutputResNet(nn.Module):
    def __init__(self, num_gender_classes=2, num_male_styles=8, num_female_styles=23, dropout_prob=0.5):
        super(MultiOutputResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # fully connected 레이어 제거

        # Dropout 레이어
        self.dropout = nn.Dropout(p=dropout_prob)

        # Batch Normalization 추가
        self.bn = nn.BatchNorm1d(in_features)

        # 성별 분류 레이어
        self.fc_gender = nn.Linear(in_features, num_gender_classes)

        # 남성 및 여성 스타일 분류 레이어
        self.fc_male_style = nn.Linear(in_features, num_male_styles)
        self.fc_female_style = nn.Linear(in_features, num_female_styles)

    def forward(self, x):
        features = self.resnet(x)
        features = self.dropout(features)
        features = self.bn(features)  # Batch Normalization 적용

        # 성별 분류
        gender_output = self.fc_gender(features)

        # 남성 및 여성 스타일 분류
        male_style_output = self.fc_male_style(features)
        female_style_output = self.fc_female_style(features)

        return gender_output, male_style_output, female_style_output


# 5. 모델 생성 및 가중치 초기화
#가중치 초기화 함수
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)  # Xavier 초기화 (Glorot 초기화)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)  # bias는 0으로 초기화

# 모델 생성 후 가중치 초기화 적용
num_male_styles = 8   # 남성 스타일 개수
num_female_styles = 23 # 여성 스타일 개수
model = MultiOutputResNet(num_gender_classes=2, num_male_styles=num_male_styles, num_female_styles=num_female_styles, dropout_prob=0.5)
model.apply(initialize_weights)  # 가중치 초기화 수행
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Focal Loss 구현
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.alpha is not None:
            at = self.alpha.gather(0, targets.data.view(-1))
            focal_loss = focal_loss * at

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


# 손실 함수 (Focal Loss 사용)
criterion_gender = FocalLoss(gamma=2)
criterion_style = FocalLoss(gamma=2)

# 옵티마이저 및 학습률 스케줄러 설정
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # 5 epoch마다 학습률 감소

# 6. 학습 과정 정의 (진행 퍼센트 표시)
def train(model, train_loader, criterion_gender, criterion_style, optimizer):
    model.train()
    running_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    train_loader_tqdm = tqdm(train_loader, desc="Training", unit="batch")

    for images, gender_labels, style_labels in train_loader_tqdm:
        images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

        optimizer.zero_grad()

        gender_output, male_style_output, female_style_output = model(images)

        # 성별 분류 손실 계산
        loss_gender = criterion_gender(gender_output, gender_labels)

        # 성별에 따라 스타일 분류 손실 계산
        loss_style = 0
        predicted_style = []
        for i in range(gender_labels.size(0)):
            if gender_labels[i] == 0:  # 여성일 때
                loss_style += criterion_style(female_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
                predicted_style.append(torch.max(female_style_output[i].unsqueeze(0), 1)[1].item())
            else:  # 남성일 때
                loss_style += criterion_style(male_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
                predicted_style.append(torch.max(male_style_output[i].unsqueeze(0), 1)[1].item())

        loss = loss_gender + loss_style  # 총 손실은 성별과 스타일 손실의 합

        # 역전파 및 최적화
        loss.backward()

        # Gradient Clipping 적용
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()

        # 성별 분류 정확도 계산
        _, predicted_gender = torch.max(gender_output, 1)
        total += gender_labels.size(0)
        correct_gender += (predicted_gender == gender_labels).sum().item()

        # 스타일 분류 정확도 계산
        correct_style += (torch.tensor(predicted_style) == style_labels).sum().item()

        # 진행 상태 표시
        train_loader_tqdm.set_postfix(
            loss=loss.item(),
            gender_acc=correct_gender / total,
            style_acc=correct_style / total  # 스타일 정확도 추가
        )

    avg_loss = running_loss / len(train_loader)
    gender_accuracy = correct_gender / total
    style_accuracy = correct_style / total
    return avg_loss, gender_accuracy, style_accuracy


def validate(model, val_loader, criterion_gender, criterion_style):
    model.eval()
    val_loss = 0.0
    correct_gender = 0
    correct_style = 0
    total = 0

    val_loader_tqdm = tqdm(val_loader, desc="Validation", unit="batch")

    with torch.no_grad():
        for images, gender_labels, style_labels in val_loader_tqdm:
            images, gender_labels, style_labels = images.to(device), gender_labels.to(device), style_labels.to(device)

            gender_output, male_style_output, female_style_output = model(images)

            loss_gender = criterion_gender(gender_output, gender_labels)
            loss_style = 0
            predicted_style = []
            for i in range(gender_labels.size(0)):
                if gender_labels[i] == 0:
                    loss_style += criterion_style(female_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
                    predicted_style.append(torch.max(female_style_output[i].unsqueeze(0), 1)[1].item())
                else:
                    loss_style += criterion_style(male_style_output[i].unsqueeze(0), style_labels[i].unsqueeze(0))
                    predicted_style.append(torch.max(male_style_output[i].unsqueeze(0), 1)[1].item())

            loss = loss_gender + loss_style
            val_loss += loss.item()

            _, predicted_gender = torch.max(gender_output, 1)
            total += gender_labels.size(0)
            correct_gender += (predicted_gender == gender_labels).sum().item()

            # 스타일 분류 정확도 계산
            correct_style += (torch.tensor(predicted_style) == style_labels).sum().item()

            val_loader_tqdm.set_postfix(
                loss=loss.item(),
                gender_acc=correct_gender / total,
                style_acc=correct_style / total  # 스타일 정확도 추가
            )

    avg_val_loss = val_loss / len(val_loader)
    gender_accuracy = correct_gender / total
    style_accuracy = correct_style / total
    return avg_val_loss, gender_accuracy, style_accuracy


def train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        avg_loss, gender_acc, style_acc = train(model, train_loader, criterion_gender, criterion_style, optimizer)
        print(f"Training - Loss: {avg_loss:.4f}, Gender Accuracy: {gender_acc:.4f}, Style Accuracy: {style_acc:.4f}")

        avg_val_loss, val_gender_acc, val_style_acc = validate(model, val_loader, criterion_gender, criterion_style)
        print(f"Validation - Loss: {avg_val_loss:.4f}, Gender Accuracy: {val_gender_acc:.4f}, Style Accuracy: {val_style_acc:.4f}")

In [ ]:
# 9. 학습 및 검증 데이터 로더 설정 후 학습 실행
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

train_model(model, train_loader, val_loader, criterion_gender, criterion_style, optimizer, num_epochs=30)

Epoch 1/30


Training: 100%|██████████| 128/128 [1:16:50<00:00, 36.02s/batch, gender_acc=0.495, loss=23.6, style_acc=0.0958]


Training - Loss: 92.3738, Gender Accuracy: 0.4953, Style Accuracy: 0.0958


Validation: 100%|██████████| 30/30 [08:10<00:00, 16.34s/batch, gender_acc=0.456, loss=52.3, style_acc=0.13]


Validation - Loss: 73.0431, Gender Accuracy: 0.4564, Style Accuracy: 0.1304
Epoch 2/30


...

Validation: 100%|██████████| 30/30 [05:09<00:00, 10.32s/batch, gender_acc=0.564, loss=47.1, style_acc=0.174]


Validation - Loss: 65.4238, Gender Accuracy: 0.5636, Style Accuracy: 0.1735
Epoch 12/30


Training: 100%|██████████| 128/128 [35:45<00:00, 16.76s/batch, gender_acc=0.51, loss=13.5, style_acc=0.129]


Training - Loss: 78.9973, Gender Accuracy: 0.5098, Style Accuracy: 0.1285


Validation:  33%|███▎      | 10/30 [01:46<03:04,  9.23s/batch, gender_acc=0.628, loss=68.4, style_acc=0.194]